In [1]:
import astroplan as ap
from astropy.coordinates import SkyCoord
from astropy.table import QTable
from astropy.time import Time
import astropy.units as u
from astroquery.gaia import Gaia

In [2]:
# # read in the text format version of the WDS and save as an Astropy QTable
# error_lines = []
# with open("wdsweb_summ2.txt", "r", newline="\n") as f:
#     for _ in range(5):
#         f.readline()
#     rows = []
#     for line in f.readlines():
#         try:
#             line = line.strip()  # remove EOL character
#             row = {}
#             row["ID"] = line[0:10]
#             row["Discoverer"] = line[10:17]
#             components_text = line[17:22]
#             if not components_text.strip() == "":
#                 row["Components"] = components_text
#             row["Num Obs"] = int(line[33:37])
#             row["Separation"] = (float(line[46:51]) + float(line[51:57])) / 2 * u.arcsec
#             row["Position Angle"] = int(line[42:45])
#             row["Mag A"] = float(line[58:63])
#             magb_text = line[63:69]
#             if not magb_text.strip() == ".":
#                 row["Mag B"] = float(magb_text)
#                 row["Mag Avg"] = (row["Mag A"] + row["Mag B"]) / 2
#             else:
#                 row["Mag Avg"] = row["Mag A"]
#             spectral_text = line[70:79]
#             if not spectral_text.strip() == "":
#                 row["Spectral Type"] = spectral_text
#             notes_text = line[107:111]
#             if not notes_text.strip() == "":
#                 row["Notes"] = notes_text
#             row["RA"] = Angle(f"{line[112:114]}h{line[114:116]}m{line[116:121]}s").to(u.deg)
#             row["Dec"] = Angle(f"{line[121:124]}d{line[124:126]}m{line[126:130]}s").to(u.deg)
#             pmra_text = line[80:84]
#             row["PM RA"] = u.Quantity(0 if pmra_text.strip() == "" else int(pmra_text), unit=u.mas / u.yr)
#             pmdec_text = line[84:88]
#             row["PM Dec"] = u.Quantity(0 if pmdec_text.strip() == "" else int(pmdec_text), unit=u.mas / u.yr)
#             rows.append(row)
#         except:
#             error_lines.append(line)
#             continue
# print(f"{len(error_lines)} errors and {len(rows)} lines successfully processed")
# print("Error lines")
# for line in error_lines:
#     print(line)

# wds = QTable(
#     rows,
#     names=[
#         "ID",
#         "Discoverer",
#         "Components",
#         "Num Obs",
#         "Separation",
#         "Position Angle",
#         "Mag A",
#         "Mag B",
#         "Mag Avg",
#         "Spectral Type",
#         "Notes",
#         "RA",
#         "Dec",
#         "PM RA",
#         "PM Dec",
#     ],
# )
# wds["Separation"].info.format = ".2f"
# wds["Mag A"].info.format = ".2f"
# wds["Mag B"].info.format = ".2f"
# wds["Mag Avg"].info.format = ".2f"
# wds["RA"].info.format = "12.8f"
# wds["Dec"].info.format = "13.8f"
# wds.write("wdsweb_summ2.csv", overwrite=True)
# wds.write("wdsweb_summ2.ecsv", overwrite=True)
# wds

In [2]:
wds = QTable.read("wdsweb_summ2.ecsv")
wds


ID,Discoverer,Components,Num Obs,Separation,Position Angle,Mag A,Mag B,Mag Avg,Spectral Type,Notes,RA,Dec,PM RA,PM Dec
,,,,arcsec,,,,,,,deg,deg,mas / yr,mas / yr
str10,str7,str5,int32,float64,int32,float64,float64,float64,str9,str4,float64,float64,float64,float64
00000+7530,A 1248,--,5,0.70,235,10.27,11.50,10.88,A7IV,--,0.02766667,75.48327778,34.0,5.0
00000+4004,ES 2543,AB,6,4.60,253,12.10,13.10,12.60,--,--,0.01525000,40.08872222,8.0,-4.0
00000+4004,ES 2543,AC,5,17.20,66,12.10,14.10,13.10,--,--,0.01525000,40.08872222,8.0,-4.0
00000+3852,BU 860,--,15,6.65,107,6.62,11.40,9.01,B9,--,0.00500000,38.85927778,3.0,-6.0
00000+0044,SKF1115,--,3,0.90,30,17.80,17.80,17.80,M1,--,359.99937500,0.72575000,-24.0,-20.0
00000-0530,OCC 622,--,0,0.10,-1,9.50,9.70,9.60,G5,--,0.00033333,-5.49436111,24.0,-12.0
00001+7727,LOC 122,--,6,8.15,249,14.10,16.60,15.35,--,T,0.03466667,77.44361111,86.0,37.0
00001+5400,ES 704,--,10,4.90,116,9.50,11.50,10.50,--,--,0.02850000,54.00005556,-30.0,3.0


# 2024-09-04 Updated procedure

As a result of the meeting, and emails with Jimmy, the following is the new procedure:

* Search the WDS for binaries, in RA 0h only, for the brightest match to the following criteria:
  * Separation under 0.5"
  * Magnitude brighter than 9
  * Magnitude difference less than 2
* Test if observable from APO during the A half of 2024-09-16.  If not, move on to the next brightest binary
* Search Gaia for a radius 110% of the separation of that binary, centered on the WDS coordinates
* If you don't get exactly 2 stars in that query, move on to the next brightest binary
* We now have Gaia's version of the binary.  Use the Gaia info for PA, separation, magnitudes, etc.
* From the midpoint of the Gaia binary, do a new Gaia search for point sources with the following criteria:
  * a 4' radius around the binary
  * within 1 mag of the brighter of the two components of the binary
* If there are no point sources found, move on to the next brightest binary
* For each of the possibly multiple point sources found, enter the binary & point source info in the targets table
* Move on to RA 1h and repeat the process
* Finish once the list of targets is a specified length, which might mean rotating through all the hours in RA multiple times

This avoids using coordinates from multiple catalogs simultaneously, and allows for the possibility of multiple point sources of varying separations for a given binary.

In [15]:
from astropy.coordinates import Angle
from astropy.table import QTable
import astropy.units as u
import numpy as np

# WDS binary candidate criteria
dimmest_binary = 9
brightest_binary = 3
# narrowest_binary = u.Quantity(0, unit=u.arcsec)
narrowest_binary = 0.5 * u.arcsec
widest_binary = 1 * u.arcsec
binary_mag_difference = 1
binary_criteria = (
    (wds["Mag Avg"] < dimmest_binary)
    & (wds["Mag Avg"] > brightest_binary)
    & (wds["Separation"] > narrowest_binary)
    & (wds["Separation"] < widest_binary)
    & (abs(wds["Mag A"] - wds["Mag B"]) < binary_mag_difference)
)

# criteria to find matching gaia stars representing that binary
binary_match_cone = 5 * u.arcsec
binary_match_mag = 1
binary_match_sep_factor = 1.5

# point source criteria
point_match_mag = 1
point_match_sep_min = 1 * u.arcmin # in reality it is 0.45', but Jimmy wants to be conservative
point_match_sep_mid = 4 * u.arcmin
point_match_sep_max = 8 * u.arcmin

# list size & observation details
num_targets = 30
observer = ap.Observer.at_site("Apache Point Observatory", timezone="US/Mountain")
observation_constraints = [
    ap.AltitudeConstraint(30 * u.deg, 80 * u.deg),
    ap.AirmassConstraint(2),
    # ap.AtNightConstraint.twilight_civil(),
]
session_beg, session_end = observer.tonight(Time("2024-09-16", format="iso", scale="utc"), horizon=-6 * u.deg)
session_end -= (session_end - session_beg) / 2 # only the A half of night

gaia_epoch_offset = Time.now() - Time("2016-01-01")

wds["Used"] = False
wds["Used"][~binary_criteria] = True
targets = []
num_close, num_far = 0, 0
counts = {"Binaries": 0, "Not found": 0, "Not observable": 0, "Mag too different": 0, "2 sources too far apart": 0, "Too many Gaia stars": 0}
hour_angle = u.Quantity(0, unit=u.deg)
while num_close < num_targets or num_far < num_targets:
    binaries = wds[
        binary_criteria
        & (wds["RA"] >= hour_angle)
        & (wds["RA"] < hour_angle + 15 * u.deg)
        & (wds["Used"] == False)]
    counts["Binaries"] += len(binaries)
    binaries.sort("Mag Avg")
    print(f"RA={Angle(hour_angle).hour:.0f}h has {len(binaries)} binaries, found {num_close} close and {num_far} far so far")
    for binary in binaries:
        print(f"Looking at {binary["ID"]}, Mag {binary["Mag A"]:.1f}", end=" ")
        wds["Used"][(wds["ID"] == binary["ID"])] = True
        wds_coord = SkyCoord(binary["RA"], binary["Dec"], unit="deg")
        observable = ap.is_observable(observation_constraints, observer, wds_coord, (session_beg, session_end))
        if not observable:
            counts["Not observable"] += 1
            print("Not observable")
            continue
        # wds_gaia_epoch = wds_coord.apply_space_motion(dt=gaia_epoch_offset)
        binary_query = (
            f"select *"#DESIGNATION, ra, dec, parallax, astrometric_n_good_obs_al, ruwe, phot_g_mean_mag "
            f"  from gaiadr3.gaia_source "
            f"  WHERE CONTAINS( "
            f"      POINT('ICRS', ra, dec), "
            f"      CIRCLE('ICRS', {wds_coord.ra.value}, {wds_coord.dec.value}, {binary_match_cone.to(u.deg).value})) = 1 "
            f"    and phot_g_mean_mag < {binary["Mag A"] + binary_match_mag} "
            f"    and phot_g_mean_mag > {binary["Mag A"] - binary_match_mag} "
        )
        binary_matches = Gaia.launch_job(binary_query).get_results()
        match len(binary_matches):
            case 0:
                counts["Not found"] += 1
                print("Not found")
                continue
            case 1:
                gaia_match = binary_matches[0]
                gaia_mag = gaia_match["phot_g_mean_mag"]
                if abs(gaia_mag - binary["Mag A"]) > binary_match_mag:
                    counts["Mag too different"] += 1
                    print("Mag too different")
                    continue
                gaia_coord = SkyCoord(gaia_match["ra"], gaia_match["dec"], unit="deg")
            case 2:
                match_0 = SkyCoord(binary_matches[0]["ra"], binary_matches[0]["dec"], unit=u.deg)
                match_1 = SkyCoord(binary_matches[1]["ra"], binary_matches[1]["dec"], unit=u.deg)
                match_separation = match_0.separation(match_1)
                if match_separation > binary["Separation"] * binary_match_sep_factor:
                    counts["2 sources too far apart"] += 1
                    print(f"Found 2 sources, but too far apart ({match_separation.to(u.arcsec)} vs {binary["Separation"]}")
                    continue
                gaia_mag = (binary_matches[0]["phot_g_mean_mag"] + binary_matches[0]["phot_g_mean_mag"]) / 2
                gaia_coord = match_0.directional_offset_by(match_0.position_angle(match_1), match_separation / 2)
            case _:
                counts["Too many Gaia stars"] += 1
                print(f"Too many Gaia stars ({len(binary_matches)})")
                continue

        # we now have the gaia version of the binary
        # look for nearby point sources of similar magnitude
        point_query = (
            f"select * "
            f"  from gaiadr3.gaia_source "
            f"  where CONTAINS( "
            f"      POINT('ICRS', ra, dec), "
            f"      CIRCLE('ICRS', {gaia_coord.ra.value}, {gaia_coord.dec.value}, {point_match_sep_max.to(u.deg).value})) = 1 "
            f"    and phot_g_mean_mag > {gaia_mag - point_match_mag} "
            f"    and phot_g_mean_mag < {gaia_mag + point_match_mag} "
        )
        point_matches = Gaia.launch_job(point_query).get_results()
        if len(point_matches) == 0:
            print("No point sources")
            continue
        target_found = False
        for point in point_matches:
            point_coord = SkyCoord(point["ra"], point["dec"], unit="deg")
            point_sep = gaia_coord.separation(point_coord)
            if point_sep < 1 * u.mas:
                print(f"Skipping gaia match of binary (of {len(point_matches)} total sources)")
                continue
            if point_sep < point_match_sep_min:
                print(f"Skipping too close point source {point_sep.to(u.arcsec)} away")
                continue
            target = {}
            target["Binary WDS ID"] = binary["ID"]
            target["Binary RA"] = gaia_coord.ra.to_string(unit=u.hour, precision=2)
            target["Binary Dec"] = gaia_coord.dec.to_string(unit=u.deg, precision=2)
            target["Binary Sep"] = binary["Separation"].to(u.arcsec)
            target["Binary PA"] = binary["Position Angle"]
            target["Binary Mag A"] = binary["Mag A"]
            target["Binary Mag B"] = binary["Mag B"]
            target["Point ID"] = point["DESIGNATION"]
            target["Point RA"] = point_coord.ra.to_string(unit=u.hour, precision=2)
            target["Point Dec"] = point_coord.dec.to_string(unit=u.deg, precision=2)
            target["Point Sep"] = f"{point_sep.dms[1]:.0f}m{point_sep.dms[2]:02.0f}s"
            target["Point PA"] = gaia_coord.position_angle(point_coord).to(u.deg)
            target["Point Mag"] = point["phot_g_mean_mag"]
            target["Offset RA"] = (gaia_coord.ra - point_coord.ra).to_string(unit=u.hour, precision=2)
            target["Offset Dec"] = (gaia_coord.ra - point_coord.ra).to_string(unit=u.deg, precision=2)
            midpoint = SkyCoord(
                (gaia_coord.ra + point_coord.ra) / 2,
                (gaia_coord.dec + point_coord.dec) / 2,
                unit="deg")
            target["Midpoint RA"] = midpoint.ra.to_string(unit=u.hour, precision=2)
            target["Midpoint Dec"] = midpoint.dec.to_string(unit=u.deg, precision=2)
            targets.append(target)
            if point_match_sep_min < point_sep <= point_match_sep_mid:
                num_close += 1
            elif point_match_sep_mid < point_sep <= point_match_sep_max:
                num_far += 1
            else:
                print("Shouldn't get here")
                print(point)
                print(point_sep).to(u.arcsec)
            print(f"Match found: {point["DESIGNATION"]}, target count={len(targets)}, {num_close} close, {num_far} far")
            target_found = True
        # if target_found:
        #     break
    if len(wds[wds["Used"] == False]) == 0:
        print(f"Exhausted WDS list with {num_close} close and {num_far} far targets found")
        print(counts)
        break
    hour_angle = u.Quantity((hour_angle.value + 15) % 360, unit=u.deg)
wds.remove_column("Used")

diffit_targets = QTable(targets)
diffit_targets["Binary Sep"].info.format = ".2f"
diffit_targets["Binary PA"].info.format = ".2f"
diffit_targets["Point PA"].info.format = ".0f"
diffit_targets["Point Mag"].info.format = ".2f"
diffit_targets.sort("Point Sep")
diffit_targets.write("DIFFIT Targets.csv", overwrite=True)
diffit_targets.write("DIFFIT Targets.ecsv", overwrite=True)
diffit_targets


RA=0h has 12 binaries, found 0 close and 0 far so far
Looking at 00093+7943, Mag 6.7 Skipping too close point source 0.4523805487874903 arcsec away
Skipping too close point source 0.4523805488973918 arcsec away
Looking at 00427-3828, Mag 6.6 Not observable
Looking at 00162+7657, Mag 7.0 Skipping too close point source 0.49012908463050375 arcsec away
Skipping too close point source 0.4901290846378745 arcsec away
Looking at 00134+2659, Mag 6.8 Not observable
Looking at 00519-4343, Mag 7.5 Not observable
Looking at 00047+3416, Mag 7.7 Not observable
Looking at 00335-5520, Mag 7.7 Not observable
Looking at 00202-3554, Mag 8.3 Not observable
Looking at 00554+3040, Mag 8.4 Skipping gaia match of binary (of 1 total sources)
Looking at 00504+5038, Mag 8.5 Skipping too close point source 0.4487481615656282 arcsec away
Skipping too close point source 0.44874816157983305 arcsec away
Looking at 00308+4732, Mag 8.5 Skipping too close point source 0.38730783940404273 arcsec away
Skipping too close p

Binary WDS ID,Binary RA,Binary Dec,Binary Sep,Binary PA,Binary Mag A,Binary Mag B,Point ID,Point RA,Point Dec,Point Sep,Point PA,Point Mag,Offset RA,Offset Dec,Midpoint RA,Midpoint Dec
,,,arcsec,,,,,,,,deg,,,,,
str10,str12,str13,float64,int32,float64,float64,str28,str12,str13,str5,float64,float32,str12,str12,str12,str13
03494+2423,3h49m21.77s,24d22m50.71s,0.85,-1.00,7.3,7.4,Gaia DR3 66746441777612416,3h49m16.83s,24d23m45.36s,1m27s,309,7.53,0h00m04.95s,0d01m14.20s,3h49m19.30s,24d23m18.03s
20450+1244,20h45m00.23s,12d43m36.51s,0.75,176.00,9.14,8.56,Gaia DR3 1755189001703743616,20h45m03.06s,12d42m08.54s,1m37s,155,8.05,-0h00m02.83s,-0d00m42.50s,20h45m01.64s,12d42m52.53s
19126+1651,19h12m34.47s,16d50m46.69s,0.55,135.00,7.11,7.95,Gaia DR3 4513495442518338688,19h12m27.67s,16d50m57.31s,1m38s,276,7.75,0h00m06.80s,0d01m41.99s,19h12m31.07s,16d50m52.00s
18222-1505,18h22m13.47s,-15d05m24.68s,0.70,71.00,8.62,9.01,Gaia DR3 4098217631616670592,18h22m04.40s,-15d05m26.77s,2m11s,269,9.00,0h00m09.07s,0d02m16.09s,18h22m08.94s,-15d05m25.73s
18571+2606,18h57m07.86s,26d05m45.26s,0.85,71.00,7.93,8.25,Gaia DR3 4534993025110720640,18h57m02.07s,26d08m39.07s,3m11s,336,8.61,0h00m05.79s,0d01m26.90s,18h57m04.96s,26d07m12.16s
18559+0323,18h55m51.23s,3d22m31.13s,0.75,357.00,8.77,9.19,Gaia DR3 4280804491022017024,18h56m03.14s,3d20m05.60s,3m50s,129,9.54,-0h00m11.91s,-0d02m58.60s,18h55m57.19s,3d21m18.36s
15116+1007,15h11m38.29s,10d07m29.98s,0.60,53.00,8.84,8.63,Gaia DR3 1168151080681928064,15h11m30.79s,10d03m45.55s,4m10s,206,9.00,0h00m07.49s,0d01m52.42s,15h11m34.54s,10d05m37.76s
03250+4013,3h24m57.89s,40d13m23.59s,0.85,115.00,8.22,8.83,Gaia DR3 236051338166315520,3h25m05.85s,40d09m03.84s,4m35s,161,9.13,-0h00m07.96s,-0d01m59.41s,3h25m01.87s,40d11m13.71s


In [34]:
total, not_observable = 0, 0
total_by_hour = {hour: 0 for hour in range(24)}
not_observable_by_hour = {hour: 0 for hour in range(24)}
for binary in wds[binary_criteria]:
    wds_coord = SkyCoord(binary["RA"], binary["Dec"], unit="deg")
    hour = int(wds_coord.ra.to(u.deg).value / 15)
    total += 1
    total_by_hour[hour] += 1
    observable = ap.is_observable(observation_constraints, observer, wds_coord, (session_beg, session_end))
    if not observable:
        not_observable += 1
        not_observable_by_hour[hour] += 1

print(f"{total:3d} total, {not_observable:3d} not observable, {(not_observable / total):.2f}")
for hour in range(24):
    print(f"{hour:2d}h   {total_by_hour[hour]:3d}   {not_observable_by_hour[hour]:3d}   {(not_observable_by_hour[hour] / total_by_hour[hour]):.2f}")


1674 total, 1023 not observable, 0.61
 0h    55    16   0.29
 1h    56    13   0.23
 2h    56    11   0.20
 3h    58    18   0.31
 4h    89    72   0.81
 5h   100    97   0.97
 6h    90    89   0.99
 7h    90    90   1.00
 8h    94    94   1.00
 9h    49    49   1.00
10h    45    45   1.00
11h    61    60   0.98
12h    47    44   0.94
13h    50    44   0.88
14h    68    39   0.57
15h    79    46   0.58
16h    65    34   0.52
17h    84    44   0.52
18h    89    29   0.33
19h   100    19   0.19
20h    80    17   0.21
21h    62    20   0.32
22h    53    11   0.21
23h    54    22   0.41


In [35]:
diffit_targets.sort("Binary Mag A")
diffit_targets

Binary WDS ID,Binary RA,Binary Dec,Binary Sep,Binary PA,Binary Mag A,Binary Mag B,Point ID,Point RA,Point Dec,Point Sep,Point PA,Point Mag,Offset RA,Offset Dec,Midpoint RA,Midpoint Dec
,,,arcsec,,,,,,,,deg,,,,,
str10,str12,str13,float64,int32,float64,float64,str28,str12,str13,str5,float64,float32,str12,str12,str12,str13
19126+1651,19h12m34.47s,16d50m46.69s,0.55,135.00,7.11,7.95,Gaia DR3 4513495442518338688,19h12m27.67s,16d50m57.31s,1m38s,276,7.75,0h00m06.80s,0d01m41.99s,19h12m31.07s,16d50m52.00s
18187-1837,18h18m43.26s,-18d37m10.77s,0.55,138.00,7.2,8.2,Gaia DR3 4096329564073656448,18h19m00.23s,-18d30m32.30s,7m46s,31,7.37,-0h00m16.97s,-0d04m14.62s,18h18m51.75s,-18d33m51.53s
03494+2423,3h49m21.77s,24d22m50.71s,0.85,-1.00,7.3,7.4,Gaia DR3 66746441777612416,3h49m16.83s,24d23m45.36s,1m27s,309,7.53,0h00m04.95s,0d01m14.20s,3h49m19.30s,24d23m18.03s
19585+3317,19h58m32.79s,33d16m39.75s,0.95,145.00,7.74,8.43,Gaia DR3 2034413209439419264,19h58m54.38s,33d21m05.54s,6m19s,45,8.67,-0h00m21.60s,-0d05m23.94s,19h58m43.59s,33d18m52.64s
19585+3317,19h58m32.79s,33d16m39.75s,0.95,145.00,7.74,8.43,Gaia DR3 2034387027316312192,19h58m12.37s,33d12m29.01s,5m58s,226,8.45,0h00m20.42s,0d05m06.28s,19h58m22.58s,33d14m34.38s
18571+2606,18h57m07.86s,26d05m45.26s,0.85,71.00,7.93,8.25,Gaia DR3 4534797415124370944,18h57m02.95s,25d59m58.02s,5m53s,191,7.99,0h00m04.91s,0d01m13.59s,18h57m05.41s,26d02m51.64s
18571+2606,18h57m07.86s,26d05m45.26s,0.85,71.00,7.93,8.25,Gaia DR3 4534993025110720640,18h57m02.07s,26d08m39.07s,3m11s,336,8.61,0h00m05.79s,0d01m26.90s,18h57m04.96s,26d07m12.16s
16326+4007,16h32m33.17s,40d06m47.95s,0.85,129.00,7.97,8.31,Gaia DR3 1332671662677270144,16h32m40.83s,39d59m14.04s,7m42s,169,9.02,-0h00m07.66s,-0d01m54.91s,16h32m37.00s,40d03m01.00s


In [14]:
target_groups = [
    ("DIFFIT Targets 1 to 4min.tui", (1 * u.arcmin, 4 * u.arcmin), "blue"),
    ("DIFFIT Targets 4 to 8min.tui", (4 * u.arcmin, 8 * u.arcmin), "red"),
]

diffit_targets["Point Sep Deg"] = [ Angle("0d" + sep) for sep in diffit_targets["Point Sep"]]
for filename, (lo, hi), color in target_groups:
    with open(filename, "w") as f:
        f.write(f"CSys=FK5; Date=2000; RotType=Object; RotAng=0; RotWrap=Nearest; Keep=Arc; dispcolor={color}\n")
        subset = diffit_targets[(lo < diffit_targets["Point Sep Deg"]) & (diffit_targets["Point Sep Deg"] <= hi)]
        for target in subset:
            ra_colons = target["Binary RA"].replace("h", ":").replace("m", ":").replace("s", "")
            dec_colons = target["Binary Dec"].replace("d", ":").replace("m", ":").replace("s", "")
            f.write(f'"{target["Binary WDS ID"]:<10s}"       {ra_colons:<14s} {dec_colons:<18s} PM=0.00000, 0.00000; Magnitude={target["Binary Mag A"]:5.2f}\n')
        print(f"Wrote {len(subset)} targets to {filename}")


Wrote 30 targets to DIFFIT Targets 1 to 4min.tui
Wrote 72 targets to DIFFIT Targets 4 to 8min.tui
